In [77]:
#this code goes from the original page to the top restaurants one by one and prints the title
#the thing is even when we click restaurants it doesnt show only restaurants.
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from pprint import pprint
import time

options = Options()
options.add_argument("--headless")


# Set up the Chrome driver
with webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=options) as driver:

    # Navigate to the TripAdvisor Restaurants page
    driver.get("https://www.tripadvisor.com/Restaurants-g189473-Thessaloniki_Thessaloniki_Region_Central_Macedonia.html")

    # accept cookies
    wait0 = WebDriverWait(driver, 10)
    cookie_button = wait0.until(EC.element_to_be_clickable((By.ID,'onetrust-accept-btn-handler')))
    cookie_button.click()

    parent_element = driver.find_element(By.ID, "EATERY_OVERVIEW")

    child_elements = parent_element.find_elements(By.CLASS_NAME, "ygtQB.Gi.o")

    restaurants = {}

    for child_element in child_elements:

        # Find the <a> element within each child element
        a_element = child_element.find_element(By.CLASS_NAME, "Lwqic.Cj.b")
        html_code = a_element.get_attribute("outerHTML")
        
        try:
            # Extract the name for restaurants with enumaration
            # Ex:
            # 1. Navarino Creperie -> Navarino Creperie
            restaurant = a_element.text.split('. ')[1]
        except IndexError:
            # There is no enumeration and we can select the whole text
            restaurant = a_element.text
            
        href = a_element.get_attribute("href")

        if restaurant in restaurants:
            continue
        restaurants[restaurant] = {'link' : href}


# time.sleep(20)
# driver.quit()

In [78]:
restaurants

{'Navarino Creperie': {'link': 'https://www.tripadvisor.com/Restaurant_Review-g189473-d12633282-Reviews-Navarino_Creperie-Thessaloniki_Thessaloniki_Region_Central_Macedonia.html'},
 'Argofageio': {'link': 'https://www.tripadvisor.com/Restaurant_Review-g189473-d877383-Reviews-Argofageio-Thessaloniki_Thessaloniki_Region_Central_Macedonia.html'},
 'Menu me...nou': {'link': 'https://www.tripadvisor.com/Restaurant_Review-g189473-d11924400-Reviews-Menu_me_nou-Thessaloniki_Thessaloniki_Region_Central_Macedonia.html'},
 'Kanoula': {'link': 'https://www.tripadvisor.com/Restaurant_Review-g189473-d966080-Reviews-Kanoula-Thessaloniki_Thessaloniki_Region_Central_Macedonia.html'},
 'Bougatsa Bantis': {'link': 'https://www.tripadvisor.com/Restaurant_Review-g189473-d6940281-Reviews-Bougatsa_Bantis-Thessaloniki_Thessaloniki_Region_Central_Macedonia.html'},
 'Judah Club': {'link': 'https://www.tripadvisor.com/Restaurant_Review-g189473-d15117133-Reviews-Judah_Club-Thessaloniki_Thessaloniki_Region_Central

In [5]:
import json

with open("restaurants.json", "w") as outfile:
    json.dump(restaurants, outfile)

In [60]:
#this code takes all info we want from the "menu" site

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException,TimeoutException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from pprint import pprint

import time
import pandas as pd

def get_reviews(restaurant, link):

    options = Options()
    options.add_argument("--headless")

    # Set up the Chrome driver
    with webdriver.Chrome(service=Service(ChromeDriverManager().install())) as driver:
        
        driver.execute_cdp_cmd('Network.setUserAgentOverride', {"userAgent":"crawler"})

        #we do a test to see that our setup is correct and we are able to launch the chrome browser.
        driver.get(link)

        # accept cookies
        wait0 = WebDriverWait(driver, 10)
        cookie_button = wait0.until(EC.element_to_be_clickable((By.ID,'onetrust-accept-btn-handler')))
        cookie_button.click()

        wait = WebDriverWait(driver, 20)  

        # Always pass the first time becasue the next_button might not exist (due to only 1 review page)
        next_button = True
        reviews_data = []

        while next_button:
            parent_node = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'listContainer.hide-more-mobile')))
            review_tabs = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "review-container")), parent_node)
            for review_tab in review_tabs:
                #review text doesnt always shown all together we click the more button
                try:
                    more_button = review_tab.find_element(By.XPATH, ".//span[@class='taLnk ulBlueLinks']")
                    driver.execute_script("arguments[0].click();", more_button)
                    time.sleep(1)
                except (NoSuchElementException, StaleElementReferenceException):
                    pass

                date = review_tab.find_element(By.CLASS_NAME, "ratingDate")

                try:
                    visit_date = review_tab.find_element(By.CLASS_NAME, "prw_rup.prw_reviews_stay_date_hsx")
                    user_name_tab = review_tab.find_element(By.CLASS_NAME, "info_text.pointer_cursor")
                    rest_info_tab = review_tab.find_element(By.CLASS_NAME, "ui_column.is-9")
                    review_title = rest_info_tab.find_element(By.CLASS_NAME, "noQuotes")
                    review = rest_info_tab.find_element(By.CLASS_NAME, "entry")
                    review_text = review.find_element(By.CLASS_NAME, "partial_entry")
                    review_rating = rest_info_tab.find_element(By.CLASS_NAME, "ui_bubble_rating")
                    
                except NoSuchElementException:
                    continue
                except StaleElementReferenceException:
                    continue
                # Extract the username text
                date_review = date.get_attribute("title").strip().replace('Reviewed ','')
                visit_date_review = visit_date.text.strip().replace('Date of visit: ', '')
                username = user_name_tab.text.strip().split('\n')[0]
                review_title = review_title.text.strip()
                review_text = review_text.text.strip()
                rating_class = review_rating.get_attribute("class")
                rating_value = int(rating_class.split("_")[-1])/10
                
                review_info = {
                        "Username": username,
                        "Review Date": date_review,
                        "Visit Date" : visit_date_review,
                        "Business": restaurant,
                        "Review Title": review_title,
                        "Review Text": review_text,
                        "Rating": rating_value
                    }
                reviews_data.append(review_info)

            try:
                buttons = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "unified.ui_pagination")))
                next_button = buttons.find_element(By.CLASS_NAME, "nav.next.ui_button.primary:not(.disabled)")
                next_button.click()
            except TimeoutException:
                next_button = None
            except NoSuchElementException:
                next_button = None
            except WebDriverException:
                next_button = None
            time.sleep(2)

        # Create a dataframe from the reviews data
        df_reviews = pd.DataFrame(reviews_data)

    return df_reviews

In [ ]:
# reviews_df = pd.DataFrame(columns=["Username", "Review Date", "Visit Date", "Business", "Review Title", "Review Text", "Rating"])
reviews_df_list = []

for key in restaurants.keys():

    print(key)
    link = restaurants[key]['link']

    df = get_reviews(key, link)
    reviews_df_list.append(df)


reviews_df = pd.concat(reviews_df_list)
reviews_df

In [67]:
reviews_df.to_csv('All_Reviews.csv',index=False)